# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from matplotlib.pyplot import imread
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

In [4]:
print(len(train_doc))
print(len(val_doc))

663
100


In [5]:
def standardize(img):
    mean = np.mean(img)
    std = np.std(img)
    img = (img - mean) / std
    return img

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,3,4,5,6,7,8,10,12,14,15,16,17,18,19,20,22,24,25,26,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,25,80,80,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = cv2.resize(image[:,20:140,:],(80,80)).astype(np.float32)
                    else:
                        image = cv2.resize(image[:,20:340,:],(80,80)).astype(np.float32)
                        
                    batch_data[folder,idx,:,:,0] = standardize(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = standardize(image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = standardize(image[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,25,80,80,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = cv2.resize(image[:,20:140,:],(80,80)).astype(np.float32)
                    else:
                        image = cv2.resize(image[:,20:340,:],(80,80)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = standardize(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = standardize(image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = standardize(image[:,:,2]) #normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size = 32 #experiment with the batch size

# training sequences = 663
# validation sequences = 100
# epochs = 40


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(25,80,80,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

2022-12-11 05:28:52.156869: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-12-11 05:28:52.156951: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14802 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1d:00.0, compute capability: 7.5


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
optimiser =  tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 25, 80, 80, 64)    5248      
                                                                 
 batch_normalization (BatchN  (None, 25, 80, 80, 64)   256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 25, 80, 80, 64)    0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 12, 40, 80, 64)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 40, 80, 128)   221312    
                                                                 
 batch_normalization_1 (Batc  (None, 12, 40, 80, 128)  5

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_948/385748685.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  Project_data/train ; batch size = 32
Epoch 1/40


2022-12-11 05:29:04.936180: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


21/21 [==============================] - ETA: 0s - loss: 3.0294 - categorical_accuracy: 0.2896Source path =  Project_data/val ; batch size = 32

Epoch 00001: saving model to model_init_2022-12-1105_28_48.396107/model-00001-3.02936-0.28959-1.53860-0.40000.h5
21/21 [==============================] - 64s 3s/step - loss: 3.0294 - categorical_accuracy: 0.2896 - val_loss: 1.5386 - val_categorical_accuracy: 0.4000 - lr: 0.0010
Epoch 2/40
21/21 [==============================] - ETA: 0s - loss: 1.7349 - categorical_accuracy: 0.4027
Epoch 00002: saving model to model_init_2022-12-1105_28_48.396107/model-00002-1.73494-0.40271-1.51246-0.38000.h5
21/21 [==============================] - 55s 3s/step - loss: 1.7349 - categorical_accuracy: 0.4027 - val_loss: 1.5125 - val_categorical_accuracy: 0.3800 - lr: 0.0010
Epoch 3/40
21/21 [==============================] - ETA: 0s - loss: 1.2563 - categorical_accuracy: 0.5279
Epoch 00003: saving model to model_init_2022-12-1105_28_48.396107/model-00003-1.25630

### Observations:

- As we can see from above model that it is overfitting. We have got categorical_accuracy: 0.8145 and val_categorical_accuracy: 0.8500.
- Currently we are using 25 frames with size 80*80 and 3 channels as an input.
- 4 convolutional layer followed by batch normalization, maxpooling.
- Later it is flattened follwoed by dropout and fed into Dense layer with 5 classification output.
- This model is giving good accuracy. Though it has trainanble parameters as 9.5 million which is good for webcam hosting as well.

### Lets add dropout after third CONVO 3D layer to reduce overfit and see if our accuracy is still good.

### Model 2: CONVO 3D

In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(25,80,80,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
#model.add(Dropout(0.25))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [15]:
optimiser =  tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 25, 80, 80, 64)    5248      
                                                                 
 batch_normalization_4 (Batc  (None, 25, 80, 80, 64)   256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 25, 80, 80, 64)    0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 12, 40, 80, 64)   0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 12, 40, 80, 128)   221312    
                                                                 
 batch_normalization_5 (Batc  (None, 12, 40, 80, 128) 

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_948/4010122893.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  Project_data/train ; batch size = 32
Epoch 1/40
21/21 [==============================] - ETA: 0s - loss: 3.3251 - categorical_accuracy: 0.2655Source path =  Project_data/val ; batch size = 32

Epoch 00001: saving model to model_init_2022-12-1105_28_48.396107/model-00001-3.32514-0.26546-1.53073-0.45000.h5
21/21 [==============================] - 51s 2s/step - loss: 3.3251 - categorical_accuracy: 0.2655 - val_loss: 1.5307 - val_categorical_accuracy: 0.4500 - lr: 0.0010
Epoch 2/40
21/21 [==============================] - ETA: 0s - loss: 1.7886 - categorical_accuracy: 0.3363
Epoch 00002: saving model to model_init_2022-12-1105_28_48.396107/model-00002-1.78863-0.33635-1.54580-0.47000.h5
21/21 [==============================] - 51s 3s/step - loss: 1.7886 - categorical_accuracy: 0.3363 - val_loss: 1.5458 - val_categorical_accuracy: 0.4700 - lr: 0.0010
Epoch 3/40
21/21 [==============================] - ETA: 0s - loss: 1.4413 - categorical_accuracy: 0.4419
Epoch 00003: saving mo

### Observations of Model 2 (CONVO-3D):

- So finally we have got very good model using CONVO 3D with below parameters:
1. We have used 25 frames of size 80*80 with 3 channels. 
2. Frame size was reduced/cropped to 80*80 to extract fine features only.
3. As we can see we got ***categorical_accuracy: 0.7195*** and ***val_categorical_accuracy: 0.7700***
4. This accuracy we got after trying multiple models with different parameters.
5. Total trainable parameters ***Trainable params: 9,439,365*** which is even good for webcam hosting.
6. This model has 4 convolutional 3D layer followed by batch normalization, maxpooling.
7. 0.25 dropout has been added after third convolution layer to reduce overfitting.
8. Later it is flattened follwoed by dropout and fed into Dense layer with 5 classification outputs.